In [76]:
import requests
import urllib.parse
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString
import ast
import matplotlib.pyplot as plt
from ipywidgets import interact_manual, widgets, fixed
import pandas as pd
import geopandas as gpd
import datetime
import saopaulo.sp_grid as gr
import time

import saopaulo.cycling_potential as cp
from bikescience.slope import plot_slope, plot_slopes, split_route

graph_hopper_url = 'https://graphhopper.com/api/1/route'
graph_hopper_key = 'bb4601e7-696d-4b91-a809-4b1b8256b2a8'

In [6]:
od_trips = pd.read_csv('../data/sao-paulo/od/OD_2017.csv')
od_trips.columns = [c[: c.index(',')] for c in od_trips.columns]
#od_trips.set_index('ID_ORDEM', inplace=True, drop=False)

In [11]:
display(od_trips)

,ZONA,MUNI_DOM,CO_DOM_X,CO_DOM_Y,ID_DOM,F_DOM,FE_DOM,DOM,CD_ENTRE,DATA,...,MODOPRIN,TIPVG,PAG_VIAG,TP_ESAUTO,VL_EST,PE_BICI,VIA_BICI,TP_ESTBICI,ID_ORDEM,route
0,1,36,333743,7394463,10001,1,15.416667,1,1,6092017,...,16,3,0,0,0.0,1,0,0,1,None
1,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,16,3,0,0,0.0,1,0,0,2,None
2,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,1,1,2,0,0.0,0,0,0,3,None
3,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,1,1,2,0,0.0,0,0,0,4,None
4,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,12,2,0,0,0.0,0,0,0,5,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183087,517,25,294618,7410518,5171418,0,85.531250,1418,1,15092018,...,16,3,0,0,0.0,1,0,0,183088,None
183088,517,25,295243,7411456,5171599,1,52.500000,1599,1,15092018,...,16,3,0,0,0.0,1,0,0,183089,None
183089,517,25,295243,7411456,5171599,0,52.500000,1599,1,15092018,...,16,3,0,0,0.0,1,0,0,183090,None
183090,517,25,295243,7411456,5171599,0,52.500000,1599,1,15092018,...,16,3,0,0,0.0,1,0,0,183091,None


In [27]:
from pyproj import Transformer

def convert_projected_to_latlon(x, y):
    transformer = Transformer.from_crs("EPSG:31983", "EPSG:4326")

    return transformer.transform(x, y)

In [ ]:
def get_route_from_coordinates (origin, destination):
    url =  graph_hopper_url + '?key=' + graph_hopper_key + '&point={},{}&point={},{}&' + \
           'vehicle=bike&elevation=true&type=json&points_encoded=false'

    result = requests.get(url.format(origin[0], origin[1],destination[0],destination[1]))
    if result.status_code == 200:
        path = result.json()['paths'][0]
        route = path['points']['coordinates']
        
        geo = LineString(route)
        return geo
    else:
        return None

In [64]:
def get_coords_transfers(row):
    x_y = [[row['CO_O_X'], row['CO_O_Y']]]
    if row['CO_T1_X'] > 0:
        x_y.append([row['CO_T1_X'], row['CO_T1_Y']])
    if row['CO_T2_X'] > 0:
        x_y.append([row['CO_T2_X'], row['CO_T2_Y']])
    if row['CO_T3_X'] > 0:
        x_y.append([row['CO_T3_X'], row['CO_T3_Y']])
    
    x_y.append([row['CO_D_X'], row['CO_D_Y']])

    coords = []
    for c in x_y:
        coords.append(convert_projected_to_latlon(c[0], c[1]))
        
    return coords

def get_routes(od_record):
    coords = get_coords_transfers(od_record)
    if (od_record['MODO1'] == 0): return []
    
    data = []
    for i in range(1, len(coords)):
        o = coords[i - 1]
        d = coords[i]
        r = get_route_from_coordinates(o, d)
        
        data.append([od_record['ID_ORDEM'], od_record['MODO' + str(i)], i, o[0], o[1], d[0], d[1], r])
        
    return data

In [75]:
od_trips['route'] = [None] * len(od_trips)

start = 0
step = 500

columns = ['ID_ORDEM', 'MODE', 'STEP', 'ORIG_LAT', 'ORIG_LON', 'DEST_LAT', 'DEST_LON', 'ROUTE']

while start < len(od_trips):
    routes = []
    end = min(start + step, len(od_trips))
    
    print ('**********************************')
    print (datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), start)
    print ('**********************************')
    print ('')
    progress = 0
    
    for i in range(start, end):
        
        if (i - start) / step >= progress:
            print (round(progress * 100), '%  - ', datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            progress += 0.05
            
        row = od_trips.iloc[i]        
        data = get_routes(row)
        routes.extend(data)
        
        if len(data) > 0 and data[0][7] == None:
            print('Erro --- ', data)
        
    gdf = gpd.GeoDataFrame(routes, crs={'init': 'epsg:4326'}, 
                            geometry='ROUTE', columns = columns)
    
    #display(gdf)
    gdf.to_file('bases/routes/routes_' + str(start) + '.shp')
    
    start = end
    break

**********************************
2023-05-01 19:42:17 0
**********************************

0 %  -  2023-05-01 19:42:17
5 %  -  2023-05-01 19:42:52
10 %  -  2023-05-01 19:43:20
15 %  -  2023-05-01 19:44:03
20 %  -  2023-05-01 19:44:40
Erro --------------------
Erro ---  [[114, 16, 1, -23.53214501825418, -46.632367283196295, -23.52421955088031, -46.61909739410609, None]]
Erro --------------------
Erro ---  [[115, 16, 1, -23.52421955088031, -46.61909739410609, -23.523378774925572, -46.62005664840607, None]]
25 %  -  2023-05-01 19:45:13
30 %  -  2023-05-01 19:45:47
35 %  -  2023-05-01 19:46:16
40 %  -  2023-05-01 19:46:49
Erro --------------------
Erro ---  [[207, 1, 1, -23.555320700710602, -46.673618672438735, -23.549230945484446, -46.63746604041647, None]]
45 %  -  2023-05-01 19:47:22


KeyboardInterrupt: 

In [58]:
# just to validate a initial test

def test_shp():
    teste = gpd.read_file('bases/routes/routes_0.shp')
    display(teste)

    fmap = gr.map_around_sp(the_grid=None,zoom=15,plot_grid=False)

    r = teste.iloc[109]
    print(r)
    r_split = split_route(r['geometry'], 'line')
    plot_slope(fmap, r['geometry'], 'line')

    r = teste.iloc[110]
    print(r)
    r_split = split_route(r['geometry'], 'line')
    plot_slope(fmap, r['geometry'], 'line')

    display(fmap)
    
# teste_shp

In [67]:
gdf = gpd.GeoDataFrame(routes, crs={'init': 'epsg:4326'}, 
                            geometry='ROUTE', columns = columns)
    
display(gdf)

,ID_ORDEM,MODE,STEP,ORIG_LAT,ORIG_LON,DEST_LAT,DEST_LON,ROUTE
0,1,16,1,-23.551678,-46.628857,-23.551495,-46.635114,"LINESTRING Z (-46.62883 -23.55186 751.57000, -..."
1,2,16,1,-23.551495,-46.635114,-23.551678,-46.628857,"LINESTRING Z (-46.63515 -23.55155 789.40000, -..."
2,3,1,1,-23.551678,-46.628857,-23.571829,-46.690237,"LINESTRING Z (-46.62883 -23.55186 751.57000, -..."
3,4,1,1,-23.571829,-46.690237,-23.525949,-46.666558,"LINESTRING Z (-46.69026 -23.57184 738.54000, -..."
4,4,4,2,-23.525949,-46.666558,-23.537903,-46.670920,"LINESTRING Z (-46.66684 -23.52593 733.50000, -..."
...,...,...,...,...,...,...,...,...
235,236,1,1,-23.552313,-46.631147,-23.502499,-46.624501,None
236,236,4,2,-23.502499,-46.624501,-23.515163,-46.617273,None
237,237,4,1,-23.515163,-46.617273,-23.516140,-46.624923,None
238,237,1,2,-23.516140,-46.624923,-23.552313,-46.631147,None


In [73]:
origin = [-23.516140, -46.624923]
destination = [-23.552313, -46.631147]

graph_hopper_key = 'd5609235-850b-44c1-a260-2913dc9a89d5'
graph_hopper_key = 'bb4601e7-696d-4b91-a809-4b1b8256b2a8'

url =  graph_hopper_url + '?key=' + graph_hopper_key + '&point={},{}&point={},{}&' + \
       'vehicle=bike&elevation=true&type=json&points_encoded=false'

result = requests.get(url.format(origin[0], origin[1],destination[0],destination[1]))
if result.status_code == 200:
    path = result.json()['paths'][0]
    route = path['points']['coordinates']

    geo = LineString(route)
    print(geo)
else:
    print('Erro --------------------')
    display(result)

LINESTRING Z (-46.624965 -23.516138 729.74, -46.624951 -23.515861 729.77, -46.624635 -23.515863 728.89, -46.624629 -23.515411 728.27, -46.624891 -23.515192 728.6900000000001, -46.624925 -23.515246 728.9400000000001, -46.625136 -23.515231 729.66, -46.625195 -23.516955 728.8, -46.625233 -23.517066 728.8099999999999, -46.62534 -23.520202 726.6900000000001, -46.625377 -23.521762 726.01, -46.62533 -23.521892 725.95, -46.625377 -23.523303 729.55, -46.625418 -23.525358 728.22, -46.625419 -23.525486 728.2, -46.625477 -23.525615 728.23, -46.625455 -23.525724 728.1799999999999, -46.625473 -23.526492 727.85, -46.625551 -23.528305 727.08, -46.625597 -23.530064 728.24, -46.625713 -23.530086 728.0700000000001, -46.625736 -23.531297 727.62, -46.62578 -23.531632 727.41, -46.625997 -23.532081 727.67, -46.626209 -23.532582 729.5599999999999, -46.62676 -23.533942 732.01, -46.627012 -23.534498 732.26, -46.629319 -23.539979 740.09, -46.629506 -23.540456 738.7, -46.630178 -23.54205 743.36, -46.630924 -23.54